In [1]:
import numpy as np
import pandas as pd
import csv
import jsonlines
from pandas.core.frame import DataFrame
import json
import requests
import urllib.parse
import sys
import execjs  # 可通过pip install PyExecJS安装，用来执行js脚本
import time
import http.client    #修改引用的模块
import hashlib        #修改引用的模块
from urllib import parse
import random
import fnmatch
import os
import random

In [22]:
def read_tsv(input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            return list(csv.reader(f, delimiter="\t", quotechar=quotechar))

In [51]:
def prepare_data(file_name):
    sentence1=[]
    sentence2=[]
    label=[]
    zh_sentence1=[]
    zh_sentence2=[]
    zh_label=[]
    with open(file_name, "r+", encoding="utf-8-sig") as f:
        for item in jsonlines.Reader(f):
            if item['language']=='en':
                sentence1.append(item['sentence1'])
                sentence2.append(item['sentence2'])
                label.append(item['gold_label'])
            if item['language']=='zh':
                zh_sentence1.append(item['sentence1'])
                zh_sentence2.append(item['sentence2'])
                zh_label.append(item['gold_label'])
    assert len(label)==len(zh_label)
    test_data = list(zip(sentence1,sentence2,zh_sentence1,zh_sentence2,label))
    test_data = [item for item in test_data if item[4]!='neutral']
    test_corpus = []
    for i in range(0,len(test_data),2):
        item1 = test_data[i]
        item2 = test_data[i+1]
        origin = item1[2]
        right = item1[0]
        if item1[4]=="contradiction":
            contradiction = item1[1]
            entailment = item2[1]
        else:
            contradiction = item2[1]
            entailment = item1[1]
        data1 = {}
        data1['sentence1']=origin
        data1['sentence2']=right
        data1['label'] = 'right'
        test_corpus.append(data1)
        data2 = {}
        data2['sentence1']=origin
        data2['sentence2']=contradiction
        data2['label'] = 'contradiction'
        test_corpus.append(data2)
        data3 = {}
        data3['sentence1']=origin
        data3['sentence2']=entailment
        data3['label'] = 'entailment'
        test_corpus.append(data3)
    return test_corpus

In [43]:
test_corpus = prepare_data()

In [3]:
sentence1=[]
sentence2=[]
label=[]
with open("XNLI-1.0\\xnli.test.jsonl", "r+", encoding="utf-8-sig") as f:
    for item in jsonlines.Reader(f):
        if item['language']=='en':
            sentence1.append(item['sentence1'])
            sentence2.append(item['sentence2'])
            label.append(item['gold_label'])

In [4]:

with open("XNLI-1.0\\xnli.test.jsonl", "r+", encoding="utf-8-sig") as f:
    for item in jsonlines.Reader(f):
        if item['language']=='zh':
            zh_sentence1.append(item['sentence1'])
            zh_sentence2.append(item['sentence2'])
            zh_label.append(item['gold_label'])

In [14]:
test_data = list(zip(sentence1,sentence2,zh_sentence1,zh_sentence2,label))

In [15]:
test_data = [item for item in test_data if item[4]!='neutral']

In [21]:
test_corpus = []
for i in range(0,len(test_data),2):
    item1 = test_data[i]
    item2 = test_data[i+1]
    origin = item1[2]
    right = item1[0]
    if item1[4]=="contradiction":
        contradiction = item1[1]
        entailment = item2[1]
    else:
        contradiction = item2[1]
        entailment = item1[1]
    test_corpus.append([origin,right,'right'])
    test_corpus.append([origin,contradiction,'contradiction'])
    test_corpus.append([origin,entailment,'entailment'])

In [3]:
sentence1=[]
sentence2=[]
label=[]
with open("multinli_1.0\\multinli_1.0_dev_mismatched.jsonl", "r+", encoding="utf-8-sig") as f:
    for item in jsonlines.Reader(f):
        if item['gold_label']=='entailment' or item['gold_label']=='contradiction':
            sentence1.append(item['sentence1'])
            sentence2.append(item['sentence2'])
            label.append(item['gold_label'])

In [4]:
len(label)

6703

In [36]:
train_data=list(zip(sentence1,sentence2,label))

In [22]:
train_data = [list(item) for item in train_data]

In [13]:
def chunks(arr, n):
    return [arr[i:i+n] for i in range(0, len(arr), n)]

In [28]:
class BaiduTranslate:
    def __init__(self,fromLang,toLang):
        self.url = "/api/trans/vip/translate"
        self.appid="20200527000473386" #申请的账号
        self.secretKey = 'MOVRJKhneZuEYCNd01Lf'#账号密码
        self.fromLang = fromLang
        self.toLang = toLang
        self.salt = random.randint(32768, 65536)

    def BdTrans(self,text):
        sign = self.appid + text + str(self.salt) + self.secretKey
        md = hashlib.md5()
        md.update(sign.encode(encoding='utf-8'))
        sign = md.hexdigest()
        myurl = self.url + \
                '?appid=' + self.appid + \
                '&q=' + parse.quote(text) + \
                '&from=' + self.fromLang + \
                '&to=' + self.toLang + \
                '&salt=' + str(self.salt) + \
                '&sign=' + sign
        try:
            httpClient = http.client.HTTPConnection('api.fanyi.baidu.com')
            httpClient.request('GET', myurl)
            response = httpClient.getresponse()
            html = response.read().decode('utf-8')
            html = json.loads(html)
            dst = html["trans_result"][0]["dst"]
            return  True , dst
        except Exception as e:
            return False , e

In [30]:
BaiduTranslate_test = BaiduTranslate('en','zh')

In [37]:
train_data

[('This site includes a list of all award winners and a searchable database of Government Executive articles.',
  'The Government Executive articles housed on the website are not able to be searched.',
  'contradiction'),
 ("uh i don't know i i have mixed emotions about him uh sometimes i like him but at the same times i love to see somebody beat him",
  'I like him for the most part, but would still enjoy seeing someone beat him.',
  'entailment'),
 ("yeah i i think my favorite restaurant is always been the one closest  you know the closest as long as it's it meets the minimum criteria you know of good food",
  'My favorite restaurants are always at least a hundred miles away from my house. ',
  'contradiction'),
 ("i don't know um do you do a lot of camping",
  'I know exactly.',
  'contradiction'),
 ("well that would be a help i wish they would do that here we have got so little landfill space left that we're going to run out before the end of this decade and it's really going to be

In [ ]:
datas = chunks(train_data,1000)
index=0
for data in datas:
    corpus=[]
    for item in data:
        item = list(item)
        source = item[0]
        text = BaiduTranslate_test.BdTrans(source)
        #print(text)
        text=text[1]
        item.append(text)
        print(source,text)
        data1={}
        data1['sentence1'] = item[3]
        data1['sentence2'] = item[0]
        data1['label'] = 'right'
        data2={}
        data2['sentence1'] = item[3]
        data2['sentence2'] = item[1]
        data2['label'] = item[2]
        corpus.append(data1)
        corpus.append(data2)
        time.sleep(1)
    all_data={}
    all_data['data'] = corpus
    file = 'outputs/'+str(index).zfill(4)+'.json' 
    index+=1
#     with open(file,'w+',encoding='utf-8') as f:
#         json.dump(all_data,f,ensure_ascii=False)

In [2]:
def get_json_file_list(data_dir):
    files = []
    for root, dir_names, file_names in os.walk(data_dir):
        for filename in fnmatch.filter(file_names, '*.json'):
            files.append(os.path.join(root, filename))
    return files

In [3]:
xnli_data=['./dev.json',
 './test.json']

In [4]:
files = get_json_file_list('data')

In [5]:
xnlis = []
for file in xnli_data:
    with open(file,'r+',encoding='utf-8') as f:
        data = json.load(f)
        #print(data)
        corpus = data['data']
        #print(corpus)
        for co in corpus:
            #print(co)
            if co:
                data1={}
                data1['sentence1'] = co['sentence1']
                data1['sentence2'] = co['sentence2']
                data1['label'] = co['label']
                xnlis.append(data1)

In [6]:
len(xnlis)

10020

In [7]:
files

['data\\0000.json',
 'data\\0001.json',
 'data\\0002.json',
 'data\\0003.json',
 'data\\0004.json',
 'data\\0005.json',
 'data\\0006.json']

In [8]:
multi_data = []
for file in files:
    with open(file,'r+',encoding='utf-8') as f:
        data = json.load(f)
        #print(data)
        corpus = data['data']
        #print(corpus)
        for co in corpus:
            #print(co)
            if co:
                data1={}
                data1['sentence1'] = co['sentence1']
                data1['sentence2'] = co['sentence2']
                data1['label'] = co['label']
                multi_data.append(data1)

In [10]:
len(multi_data)

6490

In [11]:
file_list = get_json_file_list('outputs')

In [12]:
train_data=[]
for file in file_list:
    with open(file,'r+',encoding='utf-8') as f:
        data = json.load(f)
        #print(data)
        corpus = data['data']
        #print(corpus)
        for co in corpus:
            #print(co)
            if co:
                data1={}
                data1['sentence1'] = co['sentence1']
                data1['sentence2'] = co['sentence2']
                data1['label'] = co['label']
                train_data.append(data1)

In [13]:
train_data.extend(multi_data)

In [14]:
len(train_data)

530036

In [19]:
for data in train_data:
    distribute[data['label']].append(data)

In [20]:
def get_datasets(all_data):
    random.shuffle(all_data)
    length = len(all_data)
    train_data = all_data[:int(length*0.7)]
    dev_data = all_data[int(length*0.7):int(length*0.8)]
    test_data = all_data[int(length*0.8):]
    return train_data,dev_data,test_data

In [21]:
trains=[]
devs=[]
tests=[]
for k,v in distribute.items():
    train_data,dev_data,test_data = get_datasets(v)
    trains.extend(train_data)
    devs.extend(dev_data)
    tests.extend(test_data)

In [22]:
tests.extend(xnlis)

In [23]:
len(tests)

116029

In [24]:
all_data={}
all_data['data']=trains
with open('train_v1.json','w+',encoding='utf-8') as f:
    json.dump(all_data,f,ensure_ascii=False)

In [25]:
all_data={}
all_data['data']=devs
with open('dev_v1.json','w+',encoding='utf-8') as f:
    json.dump(all_data,f,ensure_ascii=False)

In [26]:
all_data={}
all_data['data']=tests
with open('test_v1.json','w+',encoding='utf-8') as f:
    json.dump(all_data,f,ensure_ascii=False)

In [31]:
for data in test_data:
    distribute[data['label']]+=1

In [33]:
all_data={}
all_data['data']=train_data
with open('train.json','w+',encoding='utf-8') as f:
    json.dump(all_data,f,ensure_ascii=False)

In [32]:
distribute

{'contradiction': 54724, 'entailment': 53435, 'right': 98329}

In [26]:
files = get_json_file_list('data_v1')

In [27]:
class MySnow:
    def __init__(self,dataID):
        self.start = int(time.mktime(time.strptime('2018-01-01 00:00:00', "%Y-%m-%d %H:%M:%S")))
        self.last = int(time.time())
        self.countID = 0
        self.dataID = dataID    # 数据ID，这个自定义或是映射

    def get_id(self):
        # 时间差部分
        now = int(time.time())
        temp = now-self.start
        if len(str(temp)) < 9:  # 时间差不够9位的在前面补0
            length = len(str(temp))
            s = "0" * (9-length)
            temp = s + str(temp)
        if now == self.last:
            self.countID += 1   # 同一时间差，序列号自增
        else:
            self.countID = 0    # 不同时间差，序列号重新置为0
            self.last = now
        # 标识ID部分
        if len(str(self.dataID)) < 2:
            length = len(str(self.dataID))
            s = "0" * (2-length)
            self.dataID = s + str(self.dataID)
        # 自增序列号部分
        if self.countID == 99999:  # 序列号自增5位满了，睡眠一秒钟
            time.sleep(1)
        countIDdata = str(self.countID)
        if len(countIDdata) < 5:  # 序列号不够5位的在前面补0
            length = len(countIDdata)
            s = "0"*(5-length)
            countIDdata = s + countIDdata
        id = str(temp) + str(self.dataID) + countIDdata
        return id

In [28]:
all_datas=[]
indexs=[]
snow = MySnow(dataID="00")
for file in files:
    with open(file,'r+',encoding='utf-8') as f:
        data = json.load(f)
        #print(data)
        corpus = data['data']
        #print(corpus)
        for co in corpus:
            index=snow.get_id()
            indexs.append(index)
            co['unique_id']=index
            all_datas.append(co)
    all_data={}
    all_data['data']=all_datas
    with open(file,'w+',encoding='utf-8') as f:
        json.dump(all_data,f,ensure_ascii=False)
    all_datas=[]

In [11]:
len(indexs)==len(list(set(indexs)))

True